In [1]:
!pip install -U langchain langchain-community langchainhub langchain-core
!pip install -U chromadb
!pip install -U sentence-transformers
!pip install -U transformers
!pip install -U jupyter ipykernel


  Using cached chromadb-1.0.15-cp39-abi3-win_amd64.whl.metadata (7.1 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.22.1-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached opentelemetry_api-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.35.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached kubernetes-33.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mmh3-5.1.0-cp312-cp312-win_amd64.whl.metadata (16 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.2 MB 6.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.2 MB 4.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.2 MB 4.2 MB/s eta 0:00:03
   ------------ --------------------------- 3.4/11.2 MB 4.1 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.2 MB 4.1 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.2 MB 4.0 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.2 MB

In [1]:
import chromadb
from sentence_transformers import SentenceTransformer

# Connect to ChromaDB vector store
client = chromadb.PersistentClient(path="./vector_store")  # Path must match your backend config
collection = client.get_or_create_collection(name="insurance_docs")

# Count number of documents (vectors)
count = collection.count()
print(f"✅ Number of vectors in collection: {count}")

# Retrieve a few stored chunks
results = collection.query(
    query_texts=["dummy query to inspect"],
    n_results=5
)

# Display the results
for i, doc in enumerate(results["documents"][0]):
    print(f"\n🔹 Chunk #{i+1}")
    print("📄 Text:", doc)
    print("🆔 ID:", results["ids"][0][i])
    print("🧾 Metadata:", results["metadatas"][0][i])
    print("-" * 80)


✅ Number of vectors in collection: 309

🔹 Chunk #1
📄 Text: to Alcoholism, drug or substance abuse or any addictive condition and consequences thereof.. b. Any expenses for diagnostic tests, investigations / treatment taken without the psychiatrist advising the same and which is not duly supported by his prescriptions. c. Alternate treatment other than Allopathic treatment are not covered. d. For autism spectrum disorder, admissions, stays or day care treatment at specialised educational facilities are not covered. e. Out-patient Treatment for Mental Illness. 30) The Standard Exclusion under “Investigation & Evaluation (Code-Excl04) (a) Expenses related to any admission primarily for diagnostics and evaluation purposes only” are excluded even if the same requires confinement at a Hospital. SECTION D) EXCLUSIONS- STANDARD EXCLUSIONS APPLICABLE TO PART B- INTERNATIONAL COVER UNDER SECTION C) BENEFITS COVERED UNDER THE POLICY A. Applicable for Part B-I (IN-PATIENT BENEFITS FOR INTERNATIONA

In [9]:
from collections import Counter
doc_ids = [metadata["doc_id"] for metadata in results["metadatas"][0]]
print(Counter(doc_ids))


TypeError: string indices must be integers, not 'str'

In [7]:
from collections import Counter

results = collection.get(include=["metadatas"])

# Extract all stored doc_ids from metadata
doc_ids = [m.get("doc_id", "UNKNOWN") for m in results["metadatas"]]

# Count how many chunks came from each doc_id
counts = Counter(doc_ids)

# Print the counts
print("📦 Embedded Chunks Per PDF:")
for doc_id, count in counts.items():
    print(f" - {doc_id}: {count} chunks")


📦 Embedded Chunks Per PDF:
 - 68887062bddfb83a15c6604e: 59 chunks
 - 6888715abddfb83a15c66050: 105 chunks
 - 68887173bddfb83a15c66051: 3 chunks
 - 68887175bddfb83a15c66052: 37 chunks
 - 68887178bddfb83a15c66053: 43 chunks
 - 688daff9b98285b10f20b114: 25 chunks
 - 688daffeb98285b10f20b115: 37 chunks


In [5]:
import shutil
shutil.rmtree("chroma_db")  # or the path you're using


In [ ]:
!pip install Py

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings
import os
import shutil

# === 1. Start Fresh: Delete existing Chroma DB ===
if os.path.exists("chroma_db"):
    shutil.rmtree("chroma_db")

# === 2. Reinitialize Chroma DB ===
chroma_client = Client(Settings(
    persist_directory="chroma_db",
    chroma_db_impl="duckdb+parquet"
))
collection = chroma_client.create_collection(name="pdf_chunks")

# === 3. Initialize Embedding Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === 4. Text Chunking Function ===
def chunk_text(text: str, chunk_size=500, chunk_overlap=100) -> list[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_text(text)

# === 5. PDF to Text Function ===
def extract_text_from_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    return "\n".join([page.extract_text() or "" for page in reader.pages])

# === 6. Embed & Store ===
def embed_and_store(doc_id: str, texts: list[str]):
    embeddings = model.encode(texts, show_progress_bar=True).tolist()
    ids = [f"{doc_id}_{i}" for i in range(len(texts))]
    metadatas = [{"doc_id": doc_id} for _ in range(len(texts))]

    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=ids,
        metadatas=metadatas
    )
    print(f"✅ Stored {len(texts)} chunks for {doc_id}")

# === 7. Process All PDFs in Folder ===
def process_all_pdfs(folder_path: str):
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            doc_id = filename.replace(".pdf", "")
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)
            chunks = chunk_text(text)
            embed_and_store(doc_id, chunks)

# === 8. Run it ===
process_all_pdfs("files")  # Replace with your folder path

# (Optional) Save Chroma DB to disk
chroma_client.persist()


ModuleNotFoundError: No module named 'PyPDF2'